In [1]:
import os
from sys import stderr, stdout
import tempfile
import subprocess
from warnings import catch_warnings
import pandas as pd
import numpy as np
from Bio import SeqIO, SeqRecord, Seq, SearchIO, AlignIO, Phylo
from Bio.Blast import NCBIWWW, NCBIXML
import Bio.Entrez
from Bio.Phylo.TreeConstruction import DistanceCalculator,DistanceTreeConstructor
from pandas.core.frame import DataFrame
import logging
import time
import traceback

In [2]:
#Function to open fasta file of imput
def open_fasta(filename) -> SeqRecord:
    with open(filename) as handle:
        sequence_record = SeqIO.read(handle, 'fasta')
    logging.info('Opened sequence {}'.format(sequence_record.id))
    return sequence_record

In [3]:
#Function to run BLAST with taxid list
def blastp_with_list(sequence, list_taxid = [], query_size = 200):
    result_handler, result_storer = None, None
    #If list is empty run query without specific taxid
    if len(list_taxid) <1:
        result_handler = NCBIWWW.qblast('blastp', 'nr', sequence, hitlist_size=query_size) 
        result_storer = result_handler.read()
    #Prepare string of Entrez and parse it to qblast
    else:
        entrez_query = ''
        for taxid in list_taxid:
            entrez_query += 'txid{}[ORGN]'.format(taxid)
            if taxid != list_taxid[-1]:
                entrez_query += ' OR '
        result_handler = NCBIWWW.qblast('blastp', 'nr', sequence, entrez_query= entrez_query, hitlist_size=query_size)
        result_storer = result_handler.read()
    logging.info('BLASTp specifying {} taxid(s) completed'.format(len(list_taxid)))

    return result_storer

In [4]:

#Single blast query for threading
#Threading to be implemented 
def blastp_single_taxid(sequence, taxid, query_size = 20):
    entrez_query = 'txid{}[ORGN]'.format(taxid)
    result_handler = NCBIWWW.qblast('blastp', 'nr', sequence, entrez_query= entrez_query, hitlist_size=query_size)    
    result_storer = result_handler.read()
    return result_storer

In [5]:

#Function to create a temporary file from string and read it with SearchIO.read
def xml_string_to_handler(string):
    #make temporary file
    tmp = tempfile.NamedTemporaryFile(mode='a+')
    #write string
    tmp.write(string)
    handler = SearchIO.read(tmp.name, 'blast-xml')
    tmp.close()
    logging.info('Blast returned {} results'.format(len(handler)))
    return handler

In [6]:
#Creation of a dictionary with all HSPS
def blast_to_dictionary(blastresult):
    blast_dictionary = {'ID' : [], 'Description' : [], 'Seq_length' : [], 'Accession' : [], 'Bitscore' : [], 'Evalue' : [], 'Tot_aln_span':[], 'Identity' :[]}
    #Loop through results 
    for result in blastresult:
        blast_dictionary['ID'].append(result.id)
        blast_dictionary['Description'].append(result.description)
        blast_dictionary['Seq_length'].append(result.seq_len)
        blast_dictionary['Accession'].append(result.accession)
        #Store results of first HSP
        first_hsp = result.hsps[0]
        blast_dictionary['Bitscore'].append(first_hsp.bitscore)
        blast_dictionary['Evalue'].append(first_hsp.evalue)
        #Create variables to store results of multiple hsps
        all_alnspan, all_gapnum = [],[] 
        #Collect data of all hsps for each hit
        for hsp in result.hsps:
            all_alnspan.append(int(hsp.aln_span))
            all_gapnum.append(int(hsp.gap_num))
        #Calculate total alignment span and gaps to calculate identity
        tot_alnspan, tot_gapnum = int(), int()
        seq_len = int(result.seq_len) #DOUBLE CHECK 
        for span in all_alnspan:
            tot_alnspan += span
        for gap in all_gapnum:
            tot_gapnum += gap
        identity = ((tot_alnspan - tot_gapnum)/seq_len)*100
        blast_dictionary['Tot_aln_span'].append(tot_alnspan)
        blast_dictionary['Identity'].append(round(identity, 3))

    logging.info("{} entries were recorded from the BLASTp results".format(len(blast_dictionary['ID'])))
    return blast_dictionary

In [7]:
#Filter DF based on Evalue, sequence length and identity
def filter_df_blast(df:DataFrame, query:SeqRecord, evalue = 10**-10, difference_from_query = 50, identity_threshold=50) -> DataFrame:
    #Record initial dataframe length
    initial_len = len(df)

    remove_index = []
    #Check if main HSP is significant for threshold, store indexes of non-significant hits
    for i in range(len(df.index)):
        eval = df.iloc[i]['Evalue']
        if float(eval) > evalue:
            remove_index.append(str(i))
        
    #Copy DF
    df_to_return = df    
    #Remove indexes if list is not empty
    if len(remove_index)>0:
        df_to_return = df_to_return.drop(df.index[int(remove_index)])
        df_to_return = df_to_return.reset_index(drop=True)
    
    #Filter by +/- % of query length
    #Obtain query length
    query_length = len(query.seq)
    #Determine upper/lower threshold of acceptance for sequences 
    lower, upper = query_length*(difference_from_query/100), query_length*((difference_from_query + 100)/100)
    #Filter DF
    df_to_return = df_to_return[df_to_return['Seq_length'] > lower]
    df_to_return = df_to_return.reset_index(drop=True)
    df_to_return = df_to_return[df_to_return['Seq_length'] < upper]
    df_to_return = df_to_return.reset_index(drop=True)

    #Filter by identity
    df_to_return = df_to_return[df_to_return['Identity'] > identity_threshold]
    df_to_return = df_to_return.reset_index(drop=True)
    
    #Calculate how many sequences were removed
    final_len = len(df_to_return)

    logging.info("{} sequences were removed filtering the BLAST results DF, returning a DF with {} entries".format((initial_len-final_len),final_len))
    return df_to_return

In [8]:
#Retrieve all result's entries from efetch
def retrieve_all_efetch(list_of_entries, email):
    Bio.Entrez.email = email
    list_of_sequences = []
    for entry in list_of_entries:
        handler = Bio.Entrez.efetch(db='protein', id=entry, rettype = 'fasta',retmode = 'xml', retmax=1) #Returns JSON regardless
        gb_info = Bio.Entrez.read(handler, 'text')#Returns nested lists and dictionaries 
        list_of_sequences.append(gb_info)
    logging.info('{} protein entries were retrieved from NCBI protein database'.format(len(list_of_sequences)))
    return list_of_sequences

In [9]:
#Efetch to dictionary parser
def efetch_protein_to_dictionary(list_of_efetch):
    #Declare new dictionary
    dictionary = {'Accession':[],'Protein_ID':[], 'Taxid':[], 'Organism_name':[], 'Description':[], 'Seq_length':[], 'Prot_sequence':[]}
    for wrapper in list_of_efetch:
        try:
            #Cast into dictionary to avoid random exception
            result = dict(wrapper[0])
            acc_ver = result['TSeq_accver']
            accession = acc_ver.split('.')
            dictionary['Accession'].append(accession[0])
            dictionary['Protein_ID'].append(result['TSeq_accver'])
            dictionary['Taxid'].append(result['TSeq_taxid'])
            dictionary['Organism_name'].append(result['TSeq_orgname'])
            dictionary['Description'].append(result['TSeq_defline'])
            dictionary['Seq_length'].append(result['TSeq_length'])
            dictionary['Prot_sequence'].append(result['TSeq_sequence'])
        except KeyError:
            logging.error('A sequence failed parsing from EFetch')
            print('Could not parse one sequence from efetch')
    logging.info("{} sequences were parsed correctly".format(len(dictionary['Accession'])))
    return dictionary

In [10]:
def filter_df_taxon(df:DataFrame, n_of_sequences = 1) -> DataFrame:
    #Record initial length of DF
    initial_len = len(df)

    #Make a list of all retrieved taxons
    retrieved_taxids = df['Taxid'].tolist() 
    #Make list from dictionary to eliminate duplicates 
    retrieved_taxids = list(dict.fromkeys(retrieved_taxids))
        
    #Declare empty DF
    df_toreturn = pd.DataFrame()

    #For each taxon create a DF, sort and get best result to append to df_toreturn
    for taxid in retrieved_taxids:
        #Create temporary DF exclusive to taxon
        temp_df = df[df['Taxid'] == taxid]
        temp_df = temp_df.sort_values(['Evalue', 'Identity', 'Bitscore'], ascending=[True, False, False]) #('Identity', ascending=False)
        if len(temp_df) > 0:
            if len(temp_df) > n_of_sequences:
                df_toreturn = df_toreturn.append(temp_df[:n_of_sequences])
            else:
                df_toreturn = df_toreturn.append(temp_df)

    #Refactor indexes
    df_toreturn = df_toreturn.reset_index(drop=True)

    #Calculate final length and append info to logging 
    final_len = len(df_toreturn)
    logging.info("Filtering by taxon: {} unique taxid(s) collected, {} entries discarded".format(len(retrieved_taxids), (initial_len - final_len)))
    return df_toreturn


In [11]:
#Prepare string for alingment file
def fasta_for_alignment(query:SeqRecord, df:DataFrame) -> str:
    #Initialise string and add 
    string = ''
    if query.id != '<unknown id>':
        string += ">Query:{}\n".format(query.id)
    else:
        string += ">Query\n"
    string += "{}\n".format(query.seq)
    #Add all sequence IDs and aa sequence
    for i in range(len(df['Accession'])):
        #Check if protein is missing ------------------------------------------------Add to report? ERROR.LOG FILE!!!!!
        if len(df['Prot_sequence'][i]) <1:
            continue
        string += ">{}\n".format(df['Protein_ID'][i]) # {df['Organism_name'][i]} {df['Description'][i]}
        string += "{}".format(df['Prot_sequence'][i])
        #Check if is not the last element in the list 
        if i != len(df['Accession']):
            string += '\n'
    logging.info("{} sequences were prepared for alignment with the query".format(len(df)))
    return string

In [12]:
#Run mafft by saving the fasta sequences for alignment in a file and passing it to mafft 
def run_mafft_saving_file(fasta:str, mafft_directory:str, filename:str) -> str:
    #Write fasta file for alignment
    file = '{}.fasta'.format(filename)
    with open(file, 'w') as handle:
        handle.write(fasta)
    #Parse and run with mafft 
    command_list = [mafft_directory, '--distout', '{}'.format(file)]
    process = subprocess.Popen(command_list, universal_newlines= True, stdout = subprocess.PIPE, stderr = subprocess.PIPE)
    stdout, stderr = process.communicate()

    #Save standard error of mafft
    with open('aligned_mafft.stderr', 'w') as handle:
        handle.write(stderr)
    #Retrun alignment 
    return stdout

In [13]:
#Function to get protein sequence and save fasta file given an accession number
def get_fasta_from_accession(accession, email):

    #Get the efetch handler 
    Bio.Entrez.email = email
    handler = Bio.Entrez.efetch(db='protein', id=accession, rettype = 'fasta',retmode = 'xml', retmax=1) #Returns JSON regardless
    query_protein_efetch = Bio.Entrez.read(handler, 'text')#Returns nested lists and dictionaries 
    
    #Make a dictionary -- passed as list to recycle the efetch_protein_to_dictionary function
    dictionary_query = efetch_protein_to_dictionary([query_protein_efetch])

    #Make and save fasta file 
    fasta_string_query = ">{} \n{}".format(dictionary_query['Accession'][0], dictionary_query['Prot_sequence'][0])

    fasta_file_name = "{}_sequence.fasta".format(dictionary_query['Accession'][0])
    with open(fasta_file_name, 'w') as handle:
        handle.write(fasta_string_query)

    #Open sequence with open_fasta and return it 
    fasta_record_q = open_fasta(fasta_file_name)
    return fasta_record_q

In [14]:
def open_input(input, email):
    #Declare empty protein sequence
    protein_sequence = None
    #Check that the parsed string has a fasta extension, if it does pass it to open fasta function
    if input.endswith('.fas') or input.endswith('.fasta'):
        sequence = open_fasta(input)
        #Try translating the sequence, if an error is raised the sequence is already a peptide and can be returned
        try: 
            protein_sequence = sequence.translate(to_stop = True)
            logging.info('Nucleotide sequence was translated to protein')
        except Exception:
            protein_sequence = sequence
            logging.info('Protein sequence was opened')
    #If the string doesn't have an extension it will be passed to the get_fasta_from_accession function
    else:
        try:
            protein_sequence = get_fasta_from_accession(input, email)
            logging.info('Protein sequence was retrieved from NCBI protein database')
        except Exception as e:
            logging.error(traceback.format_exc())
            logging.error('The file must have a .fas or .fasta extension')

    return protein_sequence


In [15]:
def tree_from_alignment(alignment):
    calculator = DistanceCalculator('identity')
    distance_matrix = calculator.get_distance(alignment)

    constructor = DistanceTreeConstructor(calculator, 'nj')
    tree = constructor.build_tree(alignment)
    logging.info('Tree was produced with this and that method') #Placeholder to change
    return tree

In [16]:
#USER INPUTS
input_string = 'human_mx1.fas' #'sry_protein.fasta' 'QBA69874'
taxid_list = [] #['9592']
mafft_directory = r'/Users/Gioele/miniconda3/bin/mafft'
email = 'A.N.Other@example.com'
output_name = 'draft_v4_debug_2'


local_query = False
threading = False
query_size = 100

evalue_threshold = 10**-10
len_threshold = 50
identity_threshold = 50

sequences_per_taxon = 1

#Make tsv for figtree compatibility
make_tsv = False

#logging file 
logging.basicConfig(filename='{}.log'.format(output_name), filemode='w', format='%(levelname)s:%(message)s', level=logging.DEBUG) #logging refreshes every run and only displays type of message: message

#Filenames:
#Implemented
output_df = '{}_df.csv'.format(output_name)
output_alignment = '{}_alignment.fasta'.format(output_name)
output_xml_tree = '{}_xml_tree.xml'.format(output_name)
#To add
output_tree_newick = '{}_newick_tree.nwk'.format(output_name) #!!!!
output_tree_jpg = '{}_tree_image.jpg'.format(output_name)



In [17]:

#Open fasta record
fasta_record = open_input(input_string, email)
#Blast with list 
blast_results = blastp_with_list(fasta_record.seq, query_size=query_size, list_taxid=taxid_list)

In [18]:
#Blast w threading 


In [19]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!NEED TO DOUBLECHECK AND IMPLEMENT BLAST WITH THREADING 


#Parse handler to xml_string_to_handler to be able to feed it in SearchIO.read
handler_blast = xml_string_to_handler(blast_results)

#Make dictionary from handler
dictionary_blast = blast_to_dictionary(handler_blast)

#Transform dictionary into DF
results_df_blast = pd.DataFrame.from_dict(dictionary_blast)


#Filter DF by E-value / Bitscore / Identity
filtered_blast_df = filter_df_blast(results_df_blast, fasta_record, evalue=evalue_threshold, difference_from_query=len_threshold, identity_threshold=identity_threshold)

print("BLAST DF after filtering holds {} entries".format(len(results_df_blast)))



BLAST DF after filtering holds 100 entries


In [20]:
#Retrieve all results with efetch 
#Retrieve accession list
accession_list = filtered_blast_df['Accession'].tolist()
#Get results 
efetch_result = retrieve_all_efetch(accession_list, email)

# Make dictionary from efetch fasta results
dictionary_efetch = efetch_protein_to_dictionary(efetch_result)

#Transform dictionary into DF
efetch_df = pd.DataFrame.from_dict(dictionary_efetch)



#Merge blast and efetch DataFrames
left = filtered_blast_df.loc[:,['Accession', 'Seq_length', 'Evalue', 'Bitscore', 'Tot_aln_span', 'Identity']]
right = efetch_df.loc[:,['Accession', 'Protein_ID' ,'Taxid', 'Organism_name', 'Description', 'Prot_sequence']]
combined_df = pd.merge(left, right, on='Accession')


#Filter DF based on taxons
filtered_df = filter_df_taxon(combined_df, n_of_sequences=sequences_per_taxon)


#Save DF of sequences that are going to be aligned 
filtered_df.to_csv(output_df, index = False)


print('Filtered DF presents {} entries'.format(len(output_df)))


Could not parse one sequence from efetch
Could not parse one sequence from efetch
Filtered DF presents 23 entries


In [21]:

make_tsv = True

#Check if TSV output is requested. Useful for FigTree
if make_tsv:    
    tsv_df = filtered_df[['Protein_ID'] + [col for col in filtered_df.columns if col!= 'Protein_ID']]
    tsv_df.to_csv('{}.tsv'.format(output_df), sep = '\t', index = False)


In [22]:
#Prepare for multiple alingment
fasta_string_for_alignment = fasta_for_alignment(fasta_record, filtered_df)

#Run mafft alignment saving file
mafft_alignment = run_mafft_saving_file(fasta_string_for_alignment, mafft_directory, 'multiple_seq_fasta')

#Save mafft alignment
with open(output_alignment, 'w') as savefile:
    savefile.write(mafft_alignment)


#Open AlignIO from fasta file 
alignment = AlignIO.read(output_alignment, 'fasta')

#Construct tree from alignment 
#To IMPLEMENT DIFFERENT METHODS FOR TREE BUILDING 
tree = tree_from_alignment(alignment)  


#Write tree in xml
Phylo.write(tree, output_xml_tree, 'phyloxml')

Phylo.write(tree, output_tree_newick, 'newick')


1

In [23]:
#Re import newick string to feed to ETE3

In [24]:
with open(output_tree_newick, 'r') as handle:
    newick_string = handle.read()


In [25]:
newick_string



'(((((XP_017735713.1:0.00682,XP_030770463.1:0.00477)Inner52:0.00674,AFU81320.1:0.00615)Inner54:0.00101,(XP_023047606.1:0.00925,(AFU81305.1:0.00085,XP_011809449.1:0.00302)Inner53:0.00621)Inner55:0.00052)Inner56:0.00282,(((XP_012495571.1:0.07309,((XP_008066755.1:0.08765,XP_008569442.1:0.08374)Inner20:0.02040,((XP_006156437.1:0.07130,(XP_004466363.1:0.11042,(XP_010596962.1:0.06806,XP_007945509.1:0.08400)Inner18:0.02360)Inner23:0.00312)Inner27:0.00365,((XP_008143913.1:0.11893,XP_004429677.1:0.10143)Inner22:0.00548,((BAD11809.1:0.05973,XP_007172774.1:0.06269)Inner17:0.04612,((XP_036870729.1:0.01250,XP_036732799.1:0.01456)Inner14:0.05746,(((XP_029794794.1:0.03812,XP_039085427.1:0.03147)Inner12:0.01100,((XP_042811978.1:0.00151,XP_042812427.1:0.00107)Inner4:0.00868,(XP_040339814.1:0.08966,(((XP_030185366.1:0.00061,VFV35827.1:0.00196)Inner5:0.00239,(XP_043449495.1:0.00274,XP_023094481.1:0.00112)Inner6:0.00148)Inner7:0.00059,XP_026897378.1:0.00264)Inner8:0.00184)Inner9:0.00147)Inner10:0.02845)In

In [26]:
from ete3 import PhyloTree, TreeStyle

#Link tree with MSA

msa_tree = PhyloTree(newick_string, quoted_node_names=True, format=1)
msa_tree.link_to_alignment(alignment=mafft_alignment, alg_format='fasta')
msa_tree

'''leaf_x = None
for leaf in msa_tree.iter_leaves():
    print('Name: {}, Sequence: {}'.format(leaf.name, leaf.sequence))
    leaf_x=leaf'''

"leaf_x = None\nfor leaf in msa_tree.iter_leaves():\n    print('Name: {}, Sequence: {}'.format(leaf.name, leaf.sequence))\n    leaf_x=leaf"

In [27]:
msa_tree.show(tree_style=TreeStyle())

In [77]:
msa_tree.render('msa_tree.png')

{'faces': [[27.36126154826377,
   40.0,
   112.36126154826377,
   53.0,
   10,
   'XP_017735713.1'],
  [295.5, 39.5, 296.5, 53.5, 10, None],
  [48.174259318254215, 742.0, 78.17425931825422, 755.0, 115, 'Query'],
  [295.5, 741.5, 296.5, 755.5, 115, None],
  [14.752150366358705, 794.0, 98.7521503663587, 807.0, 124, 'XP_011724293.1'],
  [295.5, 793.5, 296.5, 807.5, 124, None],
  [21.109589041095887,
   859.0,
   105.10958904109589,
   872.0,
   133,
   'XP_003895511.1'],
  [295.5, 858.5, 296.5, 872.5, 133, None],
  [39.510990761388975, 664.0, 100.51099076138897, 677.0, 104, 'PNJ42385.1'],
  [295.5, 663.5, 296.5, 677.5, 104, None],
  [43.206435170436436,
   677.0,
   129.20643517043644,
   690.0,
   106,
   'NP_001266690.1'],
  [295.5, 676.5, 296.5, 690.5, 106, None],
  [47.29690984389932, 625.0, 132.29690984389933, 638.0, 99, 'XP_032030149.1'],
  [295.5, 624.5, 296.5, 638.5, 99, None],
  [24.290219815227783, 14.0, 108.29021981522779, 27.0, 6, 'XP_011809449.1'],
  [295.5, 13.5, 296.5, 27.5

In [ ]:
#IF you want to add the taxids you need to add the one of the query too
#EXCEPTION!! SORRY THE FASTA FORMAT SHOULD PRESENT PROTEIN ID, FORMAT:
#>gi|2765634|emb|Z78509.1|PPZ78509 P.pearcei 5.8S rRNA gene and ITS1 and ITS2 DNA


In [1]:
from ete3 import Tree


t = Tree( "((a,b),c);" )
t.render("mytree.png", w=183, units="mm")

{'faces': [[319.27535040778173,
   82.28746144530456,
   335.7328426968426,
   121.78544293905074,
   4,
   'c'],
  [625.3847069843147,
   42.78947995155837,
   645.1336977311878,
   82.28746144530456,
   3,
   'b'],
  [625.3847069843147,
   3.291498457812182,
   645.1336977311878,
   42.78947995155837,
   2,
   'a']],
 'node_areas': {0: [3.2914984578121826,
   3.291498457812182,
   645.1336977311878,
   121.78544293905074],
  1: [13.16599383124873,
   3.291498457812182,
   645.1336977311878,
   82.28746144530456],
  2: [319.27535040778173,
   3.291498457812182,
   645.1336977311878,
   42.78947995155837],
  3: [319.27535040778173,
   42.78947995155837,
   645.1336977311878,
   82.28746144530456],
  4: [13.16599383124873,
   82.28746144530456,
   335.7328426968426,
   121.78544293905074]},
 'nodes': [[307.7551058054391,
   95.45345527655329,
   320.9210996366878,
   108.61944910780201,
   4,
   None],
  [613.8644623819721,
   55.9554737828071,
   627.0304562132208,
   69.12146761405583

In [10]:
newick_string

"((gb|AFU81311.1|:0.00828,(((((gb|AFU81307.1|:0.00065,ref|XP_032030149.1|:0.00064)Inner40:0.00433,ref|XP_012357180.1|:0.00340)Inner42:0.01427,((gb|PNJ42385.1|:0.00057,gb|AFU81316.1|:0.00201)Inner46:0.00397,(ref|NP_001266690.1|:0.00722,((ref|NP_001288691.1|:0.00129,ref|XP_009431729.1|:-0.00000)Inner43:0.00710,(gb|AAX36236.1|:0.00133,(ref|NP_001138397.1|:0.00000,'Query:<unknown':0.00000)Inner44:0.00125)Inner45:0.00320)Inner47:0.00115)Inner48:0.00312)Inner49:0.00248)Inner50:0.00644,((ref|XP_012495571.1|:0.07309,((ref|XP_008066755.1|:0.08765,ref|XP_008569442.1|:0.08374)Inner20:0.02040,((ref|XP_006156437.1|:0.07130,(ref|XP_004466363.1|:0.11042,(ref|XP_010596962.1|:0.06806,ref|XP_007945509.1|:0.08400)Inner18:0.02360)Inner23:0.00312)Inner27:0.00365,((ref|XP_004429677.1|:0.10143,ref|XP_008143913.1|:0.11893)Inner22:0.00548,((dbj|BAD11809.1|:0.05973,ref|XP_007172774.1|:0.06269)Inner17:0.04612,((ref|XP_036732799.1|:0.01456,ref|XP_036870729.1|:0.01250)Inner14:0.05746,(((ref|XP_039085427.1|:0.03147

In [35]:
from ete3 import Tree

t = Tree(newick_string, quoted_node_names=True, format=1)
t
#t.render('my_newick_tree.png', w= 200, units='mm')

Tree node 'Inner67' (-0x7ffffffe5dc0467e)

In [18]:
from ete3 import NCBITaxa
ncbi = NCBITaxa()
descendants = ncbi.get_descendant_taxa(9592)
# to print names of the taxIDs in the descendants
print((descendants))


#Very fast after first time

[1159185, 406788, 183511, 9595, 46359]


In [26]:
#Function to check if the Taxid is a species one (so not to be parsed to the lineage)

def check_if_species(list_of_taxa):
    not_species = []
    ranks = ncbi.get_rank(list_of_taxa)
    for taxid in list_of_taxa:
        if ranks[int(taxid)] != 'species':
            not_species.append(taxid)
    return not_species

In [36]:
taxid_list = ['9592', '9606', '9597', '9593', '9600', '9601', '61853', '9546', '9544', '9541', '54180']

not_species = check_if_species(taxid_list)
print(not_species)


species = []
for taxid in not_species:
    lineage_to_search = ncbi.get_lineage(int(taxid))
    for spec in lineage_to_search:
        species.append(int(spec))

print(species)

checked_species = check_if_species(species)


ranks = ncbi.get_rank(species)


print(ranks)

['9592']
[1, 131567, 2759, 33154, 33208, 6072, 33213, 33511, 7711, 89593, 7742, 7776, 117570, 117571, 8287, 1338369, 32523, 32524, 40674, 32525, 9347, 1437010, 314146, 9443, 376913, 314293, 9526, 314295, 9604, 207598, 9592]
{1: 'no rank', 33154: 'clade', 9347: 'clade', 9604: 'family', 376913: 'suborder', 2759: 'superkingdom', 32523: 'clade', 32524: 'clade', 32525: 'clade', 9592: 'genus', 1437010: 'clade', 6072: 'clade', 314146: 'superorder', 117570: 'clade', 7711: 'phylum', 7776: 'clade', 40674: 'class', 9443: 'order', 33511: 'clade', 207598: 'subfamily', 131567: 'no rank', 1338369: 'clade', 314293: 'infraorder', 9526: 'parvorder', 314295: 'superfamily', 33208: 'kingdom', 89593: 'subphylum', 117571: 'clade', 8287: 'superclass', 33213: 'clade', 7742: 'clade'}
